# Part 1

In [23]:
import pandas as pd
from tqdm import tqdm
import networkx as nx

In [24]:
a_q=pd.read_csv('a2q',sep=' ', names=['u','v','t'])  #answers to questions
c_a=pd.read_csv('c2a',sep=' ', names=['u','v','t'])  #comments to answers
c_q=pd.read_csv('c2q',sep=' ', names=['u','v','t'])  #comments to questions

In [25]:
print(a_q.shape,c_a.shape,c_q.shape)

(17823525, 3) (25405374, 3) (20268151, 3)


In [26]:
#change time to hours
a_q['t'] = a_q['t'].div(3600)
c_a['t'] = c_a['t'].div(3600)
c_q['t'] = c_q['t'].div(3600)

#add type column to dataframes
a_q['type'] = 'a_q'
c_a['type'] = 'c_a'
c_q['type'] = 'c_q'

In [28]:
def time_maxmin(data,min,max): #Retrieve maximum and minimum timestamps
    m = data['t'].min()
    M = data['t'].max()
    if m < min:
        min = m 
    if M > max:
        max = M
    return min, max

In [35]:
def create_graph(data1,time_window,self_loops=0):
    #remove rows with u=v
    if self_loops == 0: 
        data1 = data1[data1['u'] != data1['v']]
    
    #remove rows not having time in given time interval
    data1 = data1[data1['t'].between(time_window[0], time_window[1])]
    
    G = nx.from_pandas_edgelist(data1, 'u', 'v', ['t', 'type'], create_using = nx.MultiDiGraph())
   
    return G

In [36]:
m=10**(10)
M=0
m, M = time_maxmin(a_q, m, M)
m, M = time_maxmin(c_a, m, M)
m, M = time_maxmin(c_q, m, M)
print(m, M)

338213.2991666667 404798.1744444444


In [51]:
#obviously choose a time window that's inside (m,M)
time_window = (m,m+1000)
A = create_graph(a_q,time_window)
B = create_graph(c_a,time_window)
C = create_graph(c_q,time_window)

In [52]:
#merge the graphs
G = nx.compose(A,B)
G = nx.compose(G,C)

# Part 2

## function 1 

In [80]:
def getFeatures(G):
    feats=[]
    feats.append('directed')
    feats.append(sum(G.nodes))                  #number of users
    feats.append(G.number_of_edges())           #number of answers/comments
    degrees=sum((dict(G.degree)).values())          
    feats.append(degrees/sum(G.nodes))          #average number of links per user  
    dens=nx.density(G)
    feats.append(dens)                          #density
    if dens>0.5:
        feats.append('dense')
    else:
        feats.append('sparse')
    return feats

## function 2 

It takes in input:

    A user/node
    An interval of time
    One of the following metrics: Betweeness 1, PageRank, ClosenessCentrality 3, DegreeCentrality

The output should return:

    The value of the given metric applied over the complete graph for the given interval of time

Give an explanaition regarding the features of the user based on all of the metrics (e.g. if the betweeness metric is high, what does this mean in practice, what if the betweeness is low but it has a high PageRank value, etc.)

In [ ]:
#costruisco funzione per grafi in cui è gia stato preso in considerazione l'intervallo di tempo
def dist(G, u, metric, time_window): #metric può essere 'bet','page','close','deg'
    if metric == 'deg':
        return G.degree(int(u))/(sum(G.nodes)-1)
    if metric == 'close':
        return nx.closeness_centrality(G, u=int(u), wf_improved=False)